In [ ]:
#| default_exp learning_context

# Learning Context

> Core module for managing learning context (memory -> LOs, prior cases, knowledge gaps, feedback preferences)

## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime
import json
from pathlib import Path
from wardbuddy.utils import setup_logger, load_context_safely, save_context_safely

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
logger = setup_logger(__name__)

## Learning Context Management

> Core module for managing student learning context and history

The system needs to track and handle student information for personalised education.  

This module handles:
* Tracking learning objectives
* Managing case history
* Monitoring knowledge gaps
* Customizing feedback preferences

and finally:
* Context persistence


In [ ]:
#| export
class LearningContext:
    """
    Manages the dynamic learning context for each student.
    
    Tracks:
    - Current rotation details
    - Learning objectives and progress
    - Knowledge gaps and strengths
    - Custom feedback preferences
    """
    
    def __init__(self, context_file: Optional[Path] = None):
        """
        Initialize learning context, optionally loading from file.
        
        Args:
            context_file: Optional path to saved context
        """
        # Initialize current rotation
        self.current_rotation = {
            "specialty": "",
            "start_date": None,
            "end_date": None,
            "key_focus_areas": []
        }
        
        # Initialize learning objectives list
        self.learning_objectives = []  # [{"objective": str, "status": "active"|"completed", "added": datetime}]
        
        # Initialize knowledge profile
        self.knowledge_profile = {
            "gaps": {},  # topic -> confidence level
            "strengths": [],
            "recent_progress": []  # [{topic, improvement, date}]
        }
        
        # Initialize feedback preferences
        self.feedback_preferences = []  # [{"focus": str, "active": bool}]
        
        if context_file and context_file.exists():
            self.load_context(context_file)
            
        logger.info("Learning context initialized")
    
    def update_rotation(self, rotation_details: Dict[str, Any]) -> None:
        """Update current rotation details"""
        self.current_rotation.update(rotation_details)
    
    def add_learning_objective(self, objective: str) -> None:
        """Add new learning objective"""
        self.learning_objectives.append({
            "objective": objective,
            "status": "active",
            "added": datetime.now().isoformat()
        })
    
    def complete_objective(self, objective: str) -> None:
        """Mark learning objective as completed"""
        for obj in self.learning_objectives:
            if obj["objective"] == objective and obj["status"] == "active":
                obj["status"] = "completed"
                obj["completed"] = datetime.now().isoformat()
                break
    
    def update_knowledge_gap(self, topic: str, confidence: float) -> None:
        """Update knowledge gap confidence level"""
        old_confidence = self.knowledge_profile["gaps"].get(topic)
        self.knowledge_profile["gaps"][topic] = confidence
        
        # Track progress if confidence improved
        if old_confidence and confidence > old_confidence:
            self.knowledge_profile["recent_progress"].append({
                "topic": topic,
                "improvement": confidence - old_confidence,
                "date": datetime.now().isoformat()
            })
            
            # Keep only recent progress
            self.knowledge_profile["recent_progress"] = \
                self.knowledge_profile["recent_progress"][-5:]
    
    def add_strength(self, topic: str) -> None:
        """Add identified strength"""
        if topic not in self.knowledge_profile["strengths"]:
            self.knowledge_profile["strengths"].append(topic)
    
    def toggle_feedback_focus(self, focus: str, active: bool) -> None:
        """Toggle feedback focus area"""
        # Update if exists
        for pref in self.feedback_preferences:
            if pref["focus"] == focus:
                pref["active"] = active
                return
                
        # Add if new
        self.feedback_preferences.append({
            "focus": focus,
            "active": active
        })
    
    def save_context(self, file_path: Path) -> None:
        """Save context to file"""
        context_data = {
            "current_rotation": self.current_rotation,
            "learning_objectives": self.learning_objectives,
            "knowledge_profile": self.knowledge_profile,
            "feedback_preferences": self.feedback_preferences
        }
        save_context_safely(context_data, file_path)
    
    def load_context(self, file_path: Path) -> None:
        """Load context from file"""
        try:
            context_data = load_context_safely(file_path)
            
            # Use .get() with default values to handle missing keys
            self.current_rotation = context_data.get("current_rotation", {
                "specialty": "",
                "start_date": None,
                "end_date": None,
                "key_focus_areas": []
            })
            
            # Ensure required keys exist
            for key in ["specialty", "start_date", "end_date", "key_focus_areas"]:
                if key not in self.current_rotation:
                    self.current_rotation[key] = None if key != "key_focus_areas" else []
            
            self.learning_objectives = context_data.get("learning_objectives", [])
            self.knowledge_profile = context_data.get("knowledge_profile", {
                "gaps": {},
                "strengths": [],
                "recent_progress": []
            })
            self.feedback_preferences = context_data.get("feedback_preferences", [])
            
            logger.info(f"Context loaded successfully from {file_path}")
            
        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")
            # Initialize with defaults on error
            self.__init__()

## Tests

In [ ]:
def test_learning_context():
    """Test LearningContext functionality"""
    # Create a temporary directory for tests
    import tempfile
    temp_dir = tempfile.mkdtemp()
    temp_path = Path(temp_dir) / "test_context.json"
    
    try:
        # Initialize context
        context = LearningContext()
        
        # Test rotation updates
        rotation_details = {
            "specialty": "Emergency Medicine",
            "start_date": "2025-01-01",
            "end_date": "2025-03-31",
            "key_focus_areas": ["Resuscitation", "Procedures"]
        }
        context.update_rotation(rotation_details)
        assert context.current_rotation["specialty"] == "Emergency Medicine"
        
        # Test learning objectives
        context.add_learning_objective("Improve chest pain assessment")
        assert len(context.learning_objectives) == 1
        assert context.learning_objectives[0]["status"] == "active"
        
        # Test completing objectives
        context.complete_objective("Improve chest pain assessment")
        assert context.learning_objectives[0]["status"] == "completed"
        
        # Test knowledge gaps
        context.update_knowledge_gap("ECG interpretation", 0.6)
        assert context.knowledge_profile["gaps"]["ECG interpretation"] == 0.6
        
        # Test progress tracking
        context.update_knowledge_gap("ECG interpretation", 0.8)
        assert len(context.knowledge_profile["recent_progress"]) == 1
        
        # Test strengths
        context.add_strength("History taking")
        assert "History taking" in context.knowledge_profile["strengths"]
        
        # Test feedback preferences
        context.toggle_feedback_focus("Include more ddx", True)
        assert len(context.feedback_preferences) == 1
        assert context.feedback_preferences[0]["active"] == True
        
        # Test persistence
        context.save_context(temp_path)
        assert temp_path.exists()
        
        # Test loading in new context
        new_context = LearningContext(temp_path)
        assert new_context.current_rotation["specialty"] == "Emergency Medicine"
        assert len(new_context.learning_objectives) == 1
        assert "ECG interpretation" in new_context.knowledge_profile["gaps"]
        
    finally:
        # Cleanup
        import shutil
        shutil.rmtree(temp_dir)
        
    print("All learning context tests passed!")

# Run tests
if __name__ == "__main__":
    test_learning_context()

2025-01-18 23:31:55,450 - __main__ - INFO - Learning context initialized
2025-01-18 23:31:55,457 - __main__ - INFO - Learning context initialized


All learning context tests passed!
